This notebook contains the processing code for the Mackay Dataset.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import random
import missingno as msno

from bokeh.io import output_file, show , output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,  LabelSet
from bokeh.palettes import Viridis256,all_palettes,BrBG, PiYG, RdGy, RdYlGn, YlGnBu, PuBuGn, Colorblind, Bokeh,Category20,Accent
import matplotlib.pyplot as plt

import seaborn as sn
from random import randrange
from lifelines import CoxPHFitter,KaplanMeierFitter

In [34]:
#checking the data files and common PID in the files
root = "data"
data = "A4C"
extension = ".mp4"
a4c = os.listdir(os.path.join(root,data))
pef = pd.read_csv('20200827_pEF_ECHO.csv')
pef['PatientID'] = pef['PatientID'].astype(str)
ref = pd.read_csv('20200827_rEF_ECHO.csv')
ref['PatientID'] = ref['PatientID'].astype(str)
Extracted_PID = pd.read_csv('Extracted_PIDs_cleaned.csv')
Extracted_PID['PatientID'] = Extracted_PID['PatientID'].astype(str)

print("Total video files of different Patients in a4c of is ", len(a4c))
print("Total unique Patients in 20200827_pEF_ECHO.csv is" , len(pef['PatientID'].unique()))
print("Total unique Patients in 20200827_rEF_ECHO.csv is" , len(ref['PatientID'].unique()))
print("Total unique Patients in Extracted_PIDs_cleaned.csv " , len(Extracted_PID['PatientID'].unique()),"\n")

pef_PID = set(pef['PatientID'].unique())
ref_PID = set(ref['PatientID'].unique())
Extra_PID = set(Extracted_PID['PatientID'].unique())
Total_csv_PID = pef_PID.union(ref_PID).union(Extra_PID)
Total_video_PID = set(a4c)

print("common Patients in pef_PID and Extra_PID is ", len(pef_PID.intersection(Extra_PID)))
print("common Patients in pef_PID and ref_PID is ", len(pef_PID.intersection(ref_PID)))
print("common Patients in Extra_PID and ref_PID is ", len(Extra_PID.intersection(ref_PID)), "\n")
print("total Patients in all of them is " , len(Total_csv_PID),"\n")

print("common Patients in all the csv files and video files is",len(Total_video_PID.intersection(Total_csv_PID)))

Total video files of different Patients in a4c of is  13152
Total unique Patients in 20200827_pEF_ECHO.csv is 4127
Total unique Patients in 20200827_rEF_ECHO.csv is 3705
Total unique Patients in Extracted_PIDs_cleaned.csv  7492 

common Patients in pef_PID and Extra_PID is  367
common Patients in pef_PID and ref_PID is  250
common Patients in Extra_PID and ref_PID is  111 

total Patients in all of them is  14615 

common Patients in all the csv files and video files is 12173


In [35]:
def get_train_val_dataset(MasterData):
    baseline = MasterData.loc[MasterData['Event_Name']=='Baseline'].reset_index(drop=True)
    baseline = baseline.sort_values(by=['PatientID']).reset_index(drop=True)
    after_event = MasterData.loc[MasterData['Event_Name']=='EventDay'].reset_index(drop=True)
    after_event = after_event.sort_values(by=['PatientID']).reset_index(drop=True)
    
    Dataframe = baseline.copy(deep=True)
    Dataframe['LVEF_label'] = after_event['Ef'].values

    gender = []
    for i in range(len(Dataframe)):
        if(Dataframe['gender'][i]=='Male' or Dataframe['gender'][i]==1):
            gender.append(1)
        else:
            gender.append(0)
            
    Dataframe['gender'] = gender
    Dataframe['change_in_LVEF'] = Dataframe['LVEF_label'] - Dataframe['Ef']
    Dataframe['change_in_LVEF(in%)'] = ((Dataframe['LVEF_label'] - Dataframe['Ef'])/Dataframe['Ef'])*100
    Dataframe['absolute_change_in_LVEF(in%)'] = abs(((Dataframe['LVEF_label'] - Dataframe['Ef'])/Dataframe['Ef'])*100)

    class_label = []
    for i in range(len(Dataframe)):
        if(Dataframe['change_in_LVEF(in%)'][i]<-5):
            class_label.append(1)
        elif (Dataframe['change_in_LVEF(in%)'][i]>5):
            class_label.append(2)
        else:
            class_label.append(0)

    Dataframe['class'] = class_label

    print("total class 0 is" ,len(Dataframe.loc[Dataframe['class']==0]))
    print("total class 1 is" ,len(Dataframe.loc[Dataframe['class']==1]))
    print("total class 2 is" ,len(Dataframe.loc[Dataframe['class']==2]))

    print("train set should contain",len(Dataframe)*.8)
    print("test set should contain",len(Dataframe)*.20)
    return Dataframe

# Getting the Data

In [36]:
## Correcting the datetime of all the csv files
ref['ExamDate'] = pd.to_datetime(ref['ExamDate'],dayfirst=True)
pef['ExamDate'] = pd.to_datetime(pef['ExamDate'])
Extracted_PID['ExamDate'] = pd.to_datetime(Extracted_PID['ExamDate'])

## Working with the ref and pef data and dropping columns that have lots of null values
ref = ref.drop(['folder'], axis = 1)
pef = pef.drop(['folder'], axis = 1)

masterData = pd.concat([ref,pef]).drop_duplicates().reset_index(drop=True)
print("Total Rows is ", len(masterData), " and Total Patients is ",len(masterData['PatientID'].unique()))
masterData = masterData.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)

masterData = masterData[masterData['ExamDate'].notna()]
#masterData = masterData[masterData['Ef'].notna()]
masterData = masterData.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
print("Total Rows after null removal is ", len(masterData), " and Total Patients is ",len(masterData['PatientID'].unique()))

Total Rows is  9745  and Total Patients is  7582
Total Rows after null removal is  9657  and Total Patients is  7494


In [37]:
#renmaing the columns to make it similar to Alberta
Extracted_PID = Extracted_PID.drop(['Unnamed: 0', 'folder','folder_1', 'reapit'],axis = 1)
Extracted_PID = Extracted_PID.rename({'IVS': 'Ivs', 'LVPW': 'Lvpw','LVIDd':'Lvidd','LVIDs':'Lvids',
                                     'LVEDV':'Lvedv','LVESV':'Lvesv','DT':'Dt','IVRT':'Ivrt',
                                     'mitral_e':'E','mitral_a':'A'}, axis=1)
Extracted_PID['Ef'] = (Extracted_PID['Lvedv'] - Extracted_PID['Lvesv'])/Extracted_PID['Lvedv']*100

#removing the rows that have null values in ExamDate/EF
print("Total Rows before null removal is ", len(Extracted_PID), " and Total Patients is ",len(Extracted_PID['PatientID'].unique()))
Extracted_PID = Extracted_PID[Extracted_PID['ExamDate'].notna()]
#Extracted_PID = Extracted_PID[Extracted_PID['Ef'].notna()]
Extracted_PID = Extracted_PID.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
print("Total Rows after null removal is ", len(Extracted_PID), " and Total Patients is ",len(Extracted_PID['PatientID'].unique()))
print("\nTotal rows that have null in each columns is\n", Extracted_PID.isnull().sum(axis = 0))

Total Rows before null removal is  18732  and Total Patients is  7492
Total Rows after null removal is  18705  and Total Patients is  7490

Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ivs               2904
Lvpw              2910
Lvidd             2907
Lvids             2907
Lvedv             4144
Lvesv             4137
Dt                3292
Ivrt              3504
E                 3170
A                 3072
TDI_e_lateral     8344
TDI_e_septal     11621
TDI_s_lateral    12097
TDI_s_septal     12104
LA_MAX_VOLUME    15435
LA_MIN_VOLUME    15427
TR_VELOCITY      14262
Ef                4145
dtype: int64


In [38]:
#concatenating the ref,pef with charlies data
Mackay_raw = pd.concat([masterData,Extracted_PID],sort=False).drop_duplicates().reset_index(drop=True)
print("Total Rows is ", len(Mackay_raw), " and Total Patients is ",len(Mackay_raw['PatientID'].unique()))
Mackay_raw = Mackay_raw.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
print("\nTotal rows that have null in each columns is\n", Mackay_raw.isnull().sum(axis = 0))
Mackay_raw = Mackay_raw[Mackay_raw['ExamDate'].notna()]
#removing the rows that have negative values in EF as it might be invalid
#Mackay_raw = Mackay_raw.loc[Mackay_raw['Ef']>0].reset_index(drop=True)
print("Total Rows is ", len(Mackay_raw), " and Total Patients is ",len(Mackay_raw['PatientID'].unique()))

Total Rows is  27726  and Total Patients is  14527

Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ao               19172
Av               19172
La               19172
Rv               19172
Efslope          19172
Ivs               3853
Lvpw              3859
Epss             19172
Lvidd             3856
Lvids             3856
Lvedv             5093
Lvesv             5086
Ef                5094
Hr               19172
Co               19172
Dt                4219
Ivrt              4420
E                 4107
A                 4017
TDI_e_lateral    17513
TDI_e_septal     20644
TDI_s_lateral    21120
TDI_s_septal     21127
LA_MAX_VOLUME    24457
LA_MIN_VOLUME    24449
TR_VELOCITY      23284
dtype: int64
Total Rows is  27726  and Total Patients is  14527


In [ ]:
Mackay_raw.sort_values(by=['ExamDate']).reset_index(drop=True)

In [7]:
# getting the Video Data
extension = ".json"
null_id = []
video_id = []
heart_rate = []
BSA = []
age = []
gender = []
MackayData = Mackay_raw.copy(deep=True)

for i in range(len(MackayData)):
    PID = MackayData['PatientID'][i]
    date = str(MackayData['ExamDate'][i].date())          
    data_path = os.path.join('data/A4C',str(PID))
    try:
        files = os.listdir(data_path)
        max_prob = 0
        for f in files:
            if(extension in f):
                json_path = os.path.join(data_path,f)
                with open(json_path) as f:
                    json_info = json.load(f)
                    if(json_info['study']['ed']==date):
                        if(max_prob<json_info['classification']['prob']):
                            max_prob = json_info['classification']['prob']
                            max_prob_id = json_info['id']
                            json_date = json_info['study']['ed']
                            json_heart_rate = json_info['tags']['HeartRate']
                            json_BSA = json_info['study']['BSA']
                            try:
                                json_gender = json_info['tags']['PatientSex']
                            except KeyError:
                                json_gender = json_info['study']['gender']
                            json_age = json_info['study']['age']
        if(json_date!=date):
            raise FileNotFoundError
        video_id.append(max_prob_id)
        heart_rate.append(json_heart_rate)
        age.append(json_age)
        BSA.append(json_BSA)
        if(json_gender=='M' or json_gender=='O'):
            gender.append("Male")
        elif (json_gender=='F' or json_gender=='1'):
            gender.append("Female")
        else:
            gender.append(np.nan)
            
    except FileNotFoundError:
        video_id.append(np.nan)
        heart_rate.append(np.nan)
        null_id.append(MackayData['PatientID'][i])
        BSA.append(np.nan)
        age.append(np.nan)
        gender.append(np.nan)

In [8]:
MackayData['age'] = age
MackayData['A4C_video_id'] = video_id
MackayData['Hr'] = heart_rate
MackayData['gender'] = gender
print("\nTotal rows that have null in each columns is\n", MackayData.isnull().sum(axis = 0))


Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ao               19172
Av               19172
La               19172
Rv               19172
Efslope          19172
Ivs               3853
Lvpw              3859
Epss             19172
Lvidd             3856
Lvids             3856
Lvedv             5093
Lvesv             5086
Ef                5094
Hr                6296
Co               19172
Dt                4219
Ivrt              4420
E                 4107
A                 4017
TDI_e_lateral    17513
TDI_e_septal     20644
TDI_s_lateral    21120
TDI_s_septal     21127
LA_MAX_VOLUME    24457
LA_MIN_VOLUME    24449
TR_VELOCITY      23284
age               8429
A4C_video_id      6296
gender            6453
dtype: int64


In [9]:
# getting the Video Data
extension = ".json"
null_id = []
video_id = []
heart_rate = []
BSA = []
age = []
gender = []
MackayData = MackayData.copy(deep=True)
root = "data/PLAX/data/PLAX/"
frames = 16
for i in range(len(MackayData)):
    PID = MackayData['PatientID'][i]
    date = str(MackayData['ExamDate'][i].date())          
    data_path = os.path.join(root,str(PID))
    max_prob_id = np.nan
    json_heart_rate = np.nan
    json_age = np.nan
    json_BSA = np.nan
    json_gender = np.nan
    try:
        files = os.listdir(data_path)
        max_prob = 0
        for f in files:
            if(extension in f):
                json_path = os.path.join(data_path,f)
                with open(json_path) as f:
                    json_info = json.load(f)
                    if(json_info['study']['ed']==date and json_info['n_frames']>=16):
                        if(max_prob<json_info['prob']):
                            max_prob = json_info['prob']
                            max_prob_id = json_info['id']
                            json_date = json_info['study']['ed']
                            json_heart_rate = json_info['tags']['HeartRate']
                            json_BSA = json_info['study']['BSA']
                            try:
                                json_gender = json_info['tags']['PatientSex']
                            except KeyError:
                                json_gender = json_info['study']['gender']
                            json_age = json_info['study']['age']
        if(json_date!=date):
            raise FileNotFoundError
        video_id.append(max_prob_id)
        heart_rate.append(json_heart_rate)
        age.append(json_age)
        BSA.append(json_BSA)
        if(json_gender=='M' or json_gender=='O'):
            gender.append("Male")
        elif (json_gender=='F' or json_gender=='1'):
            gender.append("Female")
        else:
            gender.append(np.nan)
            
    except FileNotFoundError:
        video_id.append(np.nan)
        heart_rate.append(np.nan)
        null_id.append(MackayData['PatientID'][i])
        BSA.append(np.nan)
        age.append(np.nan)
        gender.append(np.nan)

In [10]:
MackayData['PLAX_video_id'] = video_id
print("\nTotal rows that have null in each columns is\n", MackayData.isnull().sum(axis = 0))


Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ao               19172
Av               19172
La               19172
Rv               19172
Efslope          19172
Ivs               3853
Lvpw              3859
Epss             19172
Lvidd             3856
Lvids             3856
Lvedv             5093
Lvesv             5086
Ef                5094
Hr                6296
Co               19172
Dt                4219
Ivrt              4420
E                 4107
A                 4017
TDI_e_lateral    17513
TDI_e_septal     20644
TDI_s_lateral    21120
TDI_s_septal     21127
LA_MAX_VOLUME    24457
LA_MIN_VOLUME    24449
TR_VELOCITY      23284
age               8429
A4C_video_id      6296
gender            6453
PLAX_video_id    11812
dtype: int64


In [11]:
last_date = {}
for pid in list(MackayData['PatientID'].unique()):
    last_date[pid] = MackayData.loc[MackayData['PatientID']==pid].tail(1)['ExamDate'].iloc[0]
days_alive = []
for i in range(len(MackayData)):
    days_alive.append((last_date[MackayData["PatientID"][i]] - MackayData['ExamDate'][i]).days)
MackayData["days_alive"] = days_alive

In [12]:
MackayData.to_csv('Generated_csv_files/Mackay_all_data_Mortality.csv')
print(len(MackayData) , len(MackayData['PatientID'].unique()))

27726 14527


In [13]:
Mackay_no_null = MackayData.loc[MackayData['PLAX_video_id'].notnull()].reset_index(drop=True)
Mackay_no_null = Mackay_no_null.sort_values(by=['PatientID', 'ExamDate']).reset_index(drop=True)
print(len(Mackay_no_null))
Mackay_no_null = Mackay_no_null.drop_duplicates(subset = ['PatientID', 'ExamDate'],keep = 'last').reset_index(drop = True)
print("\nTotal rows that have null in each columns is\n", Mackay_no_null.isnull().sum(axis = 0))

15914

Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ao                9436
Av                9436
La                9436
Rv                9436
Efslope           9436
Ivs               2155
Lvpw              2156
Epss              9436
Lvidd             2158
Lvids             2158
Lvedv             2334
Lvesv             2333
Ef                2334
Hr                1585
Co                9436
Dt                2331
Ivrt              2430
E                 2262
A                 2234
TDI_e_lateral    10229
TDI_e_septal     11649
TDI_s_lateral    11933
TDI_s_septal     11934
LA_MAX_VOLUME    13151
LA_MIN_VOLUME    13147
TR_VELOCITY      13006
age               2900
A4C_video_id      1585
gender            1681
PLAX_video_id        0
days_alive           0
dtype: int64


In [14]:
len(Mackay_no_null) , len(Mackay_no_null['PatientID'].unique())

(14717, 10010)

In [15]:
Mackay_Mortality_dates = pd.read_csv('Mortality_HF_mackay.csv')
Mackay_Mortality_dates['Patient_ID'] = Mackay_Mortality_dates['Patient_ID'].astype(str)
Mackay_Mortality_dates = Mackay_Mortality_dates.rename(columns={'Patient_ID':'PatientID'})
Mackay_Mortality_dates['Mortality_Date'] = pd.to_datetime(Mackay_Mortality_dates['Mortality_Date'],dayfirst=True)
print(len(Mackay_Mortality_dates))
Mackay_Mortality_dates = Mackay_Mortality_dates.dropna(subset=['Mortality_Date'])
print(len(Mackay_Mortality_dates))
merged_Frame = pd.merge(Mackay_Mortality_dates, Mackay_no_null, on = "PatientID", how='right')
merged_Frame = merged_Frame.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
print("\nTotal rows that dont have null in each columns is\n",Mackay_Mortality_dates.notnull().sum(axis = 0))

13083
4207

Total rows that dont have null in each columns is
 PatientID         4207
Mortality_Date    4207
CV_death          1675
dtype: int64


In [16]:
len(merged_Frame) , len(merged_Frame['PatientID'].unique())    

(14717, 10010)

In [17]:
#final_data = merged_Frame.groupby(['PatientID']).tail(2)
final_data = merged_Frame
final_data = final_data.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
death_days = []
for i in range(len(final_data)):
    death_days.append((pd.to_datetime(final_data.iloc[i]['Mortality_Date'])  - pd.to_datetime(final_data.iloc[i]['ExamDate'])).days)
final_data['death_days'] = death_days
print(len(final_data))

14717


In [18]:
for i in range(len(final_data)):
    if(not final_data.iloc[i].isnull()['death_days']):
        
        #sanity checking
        if(final_data['death_days'][i] < final_data['days_alive'][i]):
            print(final_data['death_days'][i],final_data['days_alive'][i])
            
        final_data.at[i,'days_alive'] = final_data['death_days'][i]

In [19]:
#to get only those patients that have visits greater than 1 or if they have a mortality date
final_data = final_data.loc[(final_data['days_alive']>=0) | (final_data['Mortality_Date'].notnull()) ].reset_index(drop=True)
final_data = final_data.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
#final_data = final_data.drop_duplicates(subset=['PatientID'],keep='last')
final_data = final_data.reset_index(drop=True)


In [20]:
PID = list(final_data['PatientID'].unique())
visits = {}
for pid in PID:
    visits[pid] = len(final_data.loc[final_data['PatientID']==pid])

rows_done = visits.copy()
final_data = final_data.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
#to get the duration in days between each visit
n = len(final_data)
i = 0
visit_PID = []
zero_ID = []
while(i!=n):
    Total_visits = visits[final_data['PatientID'][i]]
    if(Total_visits==1):
        visit_PID.append(0)
        visits[final_data['PatientID'][i]] -=1
        i +=1
    else:
        visit_diff = (final_data['ExamDate'][i+1] - final_data['ExamDate'][i]).days
        visit_PID.append(visit_diff)
        visits[final_data['PatientID'][i]] -=1
        i +=1
        
#Sanity check 
n = 0
visits_data_PID = []
for key,value in rows_done.items():
    n += value
    if(visit_PID[n-1]!=0):
        print(key,"This key dosent have the correct visit")

    #to make a column that contains the total visits in the masterdata
    for i in range(value):
        visits_data_PID.append(value)
        
final_data["visit_duration"] = visit_PID
final_data["visits"] = visits_data_PID

In [21]:
root = "data/PLAX/data/PLAX/"

<b> Final Summary

In [22]:
print("the root data to PLAX is : ", root)
plax_video_id = set(os.listdir(root))
print("Total Patient Id's in video dataset", len(plax_video_id))
mackay_GT_id = set(MackayData['PatientID'].values)
print
print("Total Patient Id's in ground truth Dataset : ", len(mackay_GT_id))
mackay_GT_no_null_PLAX_id = set(Mackay_no_null['PatientID'].values)
print("Total Patient Id's in ground truth Dataset that only have a plax video : ", len(mackay_GT_no_null_PLAX_id))
Mackay_Mortality_id = set(Mackay_Mortality_dates['PatientID'].values)
print("Total Patient Id's in patients with mortality dataset : ", len(Mackay_Mortality_id))
final_data_id = set(final_data['PatientID'].values)
print("total Patient Id's with  a survial date or if they have a mortality : ", len(final_data_id))
print("Total matching patients between patients with all cause mortality and the ground truth dataset ID's that have a PLAX view : " ,len(mackay_GT_no_null_PLAX_id.intersection(Mackay_Mortality_id)),"/", len(mackay_GT_no_null_PLAX_id))
print("Total matching patients between patients with all cause mortality and the ground truth dataset ID's that have a PLAX view and taking only the valid ID's that have a survial date: " ,len(final_data_id.intersection(Mackay_Mortality_id)),"/", len(final_data_id))

the root data to PLAX is :  data/PLAX/data/PLAX/
Total Patient Id's in video dataset 10899
Total Patient Id's in ground truth Dataset :  14527
Total Patient Id's in ground truth Dataset that only have a plax video :  10010
Total Patient Id's in patients with mortality dataset :  4207
total Patient Id's with  a survial date or if they have a mortality :  10010
Total matching patients between patients with all cause mortality and the ground truth dataset ID's that have a PLAX view :  1068 / 10010
Total matching patients between patients with all cause mortality and the ground truth dataset ID's that have a PLAX view and taking only the valid ID's that have a survial date:  1068 / 10010


In [24]:
len(final_data), len( set(final_data['PatientID'].values))

(14717, 10010)

In [29]:
data_m = final_data.loc[~((final_data['days_alive']<=0) & (final_data['death_days'].isnull()))]

In [27]:
data_m = final_data.loc[~((final_data['days_alive']<=0) & (final_data['death_days'].isnull()))]
data_m = data_m.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
data_m = data_m.drop_duplicates(subset='PatientID', keep="first").reset_index()

In [31]:
len(data_m),len( set(data_m['PatientID'].values))

(6821, 3925)

In [62]:
final_data.to_csv('Generated_csv_files/Mackay_mortality_all_PLAX.csv')

In [32]:
time_to_take = 365
final_data_df = final_data.loc[((final_data['days_alive']>=time_to_take)) |((final_data['death_days']<=time_to_take))].reset_index(drop=True)
mortality = []
for i in range(len(final_data_df)):
    if (final_data_df['days_alive'][i]>=time_to_take):
        mortality.append(0)
    elif (final_data_df['death_days'][i]<=time_to_take):
        mortality.append(1)
final_data_df['mortality'] = mortality
final_data_df = final_data_df.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
print("total number of visits", len(final_data_df))
print("Total Patients who are dead within " + str(time_to_take) + " days", len(final_data_df.loc[final_data_df['mortality']==1]))
print("Total Patients who are alive within " + str(time_to_take) + " days", len(final_data_df.loc[final_data_df['mortality']==0]))
print(len(final_data_df.loc[final_data_df['mortality']==1]['PatientID'].unique()) , len(final_data_df.loc[final_data_df['mortality']==1]))
print(len(final_data_df.loc[final_data_df['mortality']==0]['PatientID'].unique()) , len(final_data_df.loc[final_data_df['mortality']==0]))
print(len(final_data_df),len(final_data_df['PatientID'].unique()))
death_pid = list(final_data_df.loc[final_data_df['mortality']==1]['PatientID'].unique())
data_death_pid  = final_data_df[final_data_df['PatientID'].isin(death_pid)]
data_death_pid = data_death_pid.drop_duplicates(subset='PatientID', keep="last").reset_index()
final_data_df = final_data_df[~final_data_df['PatientID'].isin(death_pid)]
frames = [final_data_df, data_death_pid]
result = pd.concat(frames,sort=False)
print("After correction and removing the mortality patients in the alive set")
print(len(result.loc[result['mortality']==1]['PatientID'].unique()) , len(result.loc[result['mortality']==1]))
print(len(result.loc[result['mortality']==0]['PatientID'].unique()) , len(result.loc[result['mortality']==0]))

total number of visits 6083
Total Patients who are dead within 365 days 589
Total Patients who are alive within 365 days 5494
539 589
3160 5494
6083 3626
After correction and removing the mortality patients in the alive set
539 539
3087 5355


In [68]:
result.to_csv('Generated_csv_files/Mackay_mortality_within_' + str(time_to_take) + '_days.csv')

# Getting the Ejection Fraction values from the data

In [29]:
len(MackayData)

27726

In [34]:
Mackay_raw = MackayData.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
PID = list(Mackay_raw['PatientID'].unique())
visits = {}
for pid in PID:
    visits[pid] = len(Mackay_raw.loc[Mackay_raw['PatientID']==pid])

rows_done = visits.copy()
final_data = Mackay_raw.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
#to get the duration in days between each visit
n = len(Mackay_raw)
i = 0
visit_PID = []
zero_ID = []
while(i!=n):
    Total_visits = visits[Mackay_raw['PatientID'][i]]
    if(Total_visits==1):
        visit_PID.append(0)
        visits[Mackay_raw['PatientID'][i]] -=1
        i +=1
    else:
        visit_diff = (Mackay_raw['ExamDate'][i+1] - Mackay_raw['ExamDate'][i]).days
        visit_PID.append(visit_diff)
        visits[Mackay_raw['PatientID'][i]] -=1
        i +=1
        
#Sanity check 
n = 0
visits_data_PID = []
for key,value in rows_done.items():
    n += value
    if(visit_PID[n-1]!=0):
        print(key,"This key dosent have the correct visit")

    #to make a column that contains the total visits in the masterdata
    for i in range(value):
        visits_data_PID.append(value)
        
Mackay_raw["visit_duration"] = visit_PID
Mackay_raw["visits"] = visits_data_PID

In [35]:
Mackay_raw = Mackay_raw.loc[Mackay_raw['visits']>1]        
Mackay_raw = Mackay_raw.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
time_to_visit = 365
PID_visit_ID = []
Mackay_2_visits = pd.DataFrame(columns = Mackay_raw.columns)
visit_type = []
for i in range(len(Mackay_raw)):
    if(Mackay_raw["visit_duration"][i]<=540 and Mackay_raw["visit_duration"][i]>=180):
        Mackay_2_visits = Mackay_2_visits.append(Mackay_raw.iloc[i])
        Mackay_2_visits = Mackay_2_visits.append(Mackay_raw.iloc[i+1])
        
        visit_type.append("Baseline")
        visit_type.append("EventDay")
        
        #to make a visit_ID token to keep track of each visit to make baselinea and 12-24 month visits rows
        visit_ID = randrange(100000000000)
        PID_visit_ID.append(visit_ID)
        PID_visit_ID.append(visit_ID)
Mackay_2_visits["Event_Name"] = visit_type
Mackay_2_visits["visit_ID"] = PID_visit_ID
Mackay_2_visits = Mackay_2_visits.drop_duplicates().reset_index(drop=True)
Mackay_2_visits = Mackay_2_visits.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
print("Total number of rows in baseline" , len(Mackay_2_visits.loc[Mackay_2_visits['Event_Name']=='Baseline']))
print("Total number of rows in " + str(time_to_take) + " days Event_Name is " , len(Mackay_2_visits.loc[Mackay_2_visits['Event_Name']=='EventDay']))
Mackay_2_visits = Mackay_2_visits.drop(['visits'], axis = 1)
print("\nTotal rows that have null in each columns is\n", Mackay_2_visits.isnull().sum(axis = 0))

Total number of rows in baseline 4759
Total number of rows in 365 days Event_Name is  4759

Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ao                8078
Av                8078
La                8078
Rv                8078
Efslope           8078
Ivs               1355
Lvpw              1351
Epss              8078
Lvidd             1350
Lvids             1350
Lvedv             1645
Lvesv             1644
Ef                1645
Hr                2445
Co                8078
Dt                1587
Ivrt              1697
E                 1539
A                 1431
TDI_e_lateral     5678
TDI_e_septal      7138
TDI_s_lateral     7374
TDI_s_septal      7378
LA_MAX_VOLUME     8915
LA_MIN_VOLUME     8915
TR_VELOCITY       8627
age               3420
A4C_video_id      2445
gender            2502
PLAX_video_id     4601
days_alive           0
visit_duration       0
Event_Name           0
visit_ID             0
dtype: int64


In [37]:
len(Mackay_2_visits)

9518

In [38]:
#removing the rows have null values in video or negative values in EF
Mackay_no_null_ef = Mackay_2_visits.loc[Mackay_2_visits['Ef'] > 0].reset_index(drop=True)

baseline_no_null = Mackay_no_null_ef.loc[Mackay_no_null_ef['Event_Name']=='Baseline'].reset_index(drop=True)
after_event_no_null = Mackay_no_null_ef.loc[Mackay_no_null_ef['Event_Name']=='EventDay'].reset_index(drop=True)

#due to the removal of some rows ,some patient ID's have only visit instead of 2 ,hence they all need to be removed
baseline_PID = set(baseline_no_null['visit_ID'].unique())
after_event_PID = set(after_event_no_null['visit_ID'].unique())
print("overlapping PID's is", len(baseline_PID.union(after_event_PID)))
print("matching PID's is", len(baseline_PID.intersection(after_event_PID)))
print("PID's that dont have a study at the event day", len(baseline_PID - after_event_PID))
print("PID's that dont have a baseline", len(after_event_PID - baseline_PID))

total_PID_no_study = list(baseline_PID - after_event_PID) + list(after_event_PID - baseline_PID)
print("PID's that dont have a study after event day is or baseline removed due to null is ", len(total_PID_no_study))

Mackay_no_null_ef = Mackay_no_null_ef.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
df_with_event = pd.DataFrame()
for i in range(len(Mackay_no_null_ef)):
    if(not (Mackay_no_null_ef['visit_ID'][i] in total_PID_no_study)):
        df_with_event = df_with_event.append(Mackay_no_null_ef[i:i+1])
        
df_with_event = df_with_event.reset_index(drop=True)

masterdata = df_with_event
masterdata = masterdata.sort_values(by=['PatientID','ExamDate']).reset_index(drop=True)
baseline = masterdata.loc[masterdata['Event_Name']=='Baseline'].reset_index(drop=True)
after_event_months = masterdata.loc[masterdata['Event_Name']=='EventDay'].reset_index(drop=True)

#sanity check
baseline_PID = set(baseline['visit_ID'].unique())
after_event_PID = set(after_event_months['visit_ID'].unique())



print("\n overlapping PID's is", len(baseline_PID.union(after_event_PID)))
print("matching PID's is", len(baseline_PID.intersection(after_event_PID)))
print("PID's that dont have a study at the event day", len(baseline_PID - after_event_PID))
print("PID's that dont have a baseline", len(after_event_PID - baseline_PID))
print("\nTotal rows that have null in each columns is\n", masterdata.isnull().sum(axis = 0))


overlapping PID's is 4522
matching PID's is 3011
PID's that dont have a study at the event day 559
PID's that dont have a baseline 952
PID's that dont have a study after event day is or baseline removed due to null is  1511

 overlapping PID's is 3011
matching PID's is 3011
PID's that dont have a study at the event day 0
PID's that dont have a baseline 0

Total rows that have null in each columns is
 PatientID            0
ExamDate             0
Ao                5045
Av                5045
La                5045
Rv                5045
Efslope           5045
Ivs                  3
Lvpw                 0
Epss              5045
Lvidd                0
Lvids                0
Lvedv                0
Lvesv                0
Ef                   0
Hr                1308
Co                5045
Dt                 220
Ivrt               312
E                  180
A                   54
TDI_e_lateral     3099
TDI_e_septal      4180
TDI_s_lateral     4363
TDI_s_septal      4367
LA_MAX_VOLUME     545

In [39]:
DataFrame = get_train_val_dataset(masterdata)
DataFrame.to_csv('Generated_csv_files/Mackay_EF_within_' + str(time_to_visit) + '_days_with_' + str(time_to_take) + '_mortality.csv')

total class 0 is 1003
total class 1 is 987
total class 2 is 1021
train set should contain 2408.8
test set should contain 602.2


In [51]:
len(total_dead)

10

In [52]:
len(total_alive)

101